# December 19, 2022
https://adventofcode.com/2022/day/19

In [ ]:
# 1267 too low

In [35]:
from math import ceil
import re

In [ ]:
def create_blueprint( orebot, claybot, obsbot_ore, obsbot_clay, geobot_ore, geobot_obs ):
    return {
        'orebot': {'ore': orebot},
        'claybot': {'ore': claybot},
        'obsbot': {'ore': obsbot_ore, 'clay': obsbot_clay},
        'geobot': {'ore': geobot_ore, 'obs': geobot_obs}
        }

test_bps = [
    create_blueprint( 4, 2, 3,14,  2,7 ),
    create_blueprint( 2, 3,  3,8, 3,12 )
]

In [72]:
fn = "data/19.txt"
puz_bps = []

with open(fn, "r") as file:
    while True:
        line = file.readline().strip("\n")
        if not line:
            break
        ma = re.search( r"\d+.*(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+", line )
        specs = []
        for i in range(1,7):
            specs.append(int(ma.group(i)))
        puz_bps.append( create_blueprint( *specs ) ) 

In [90]:
x = 12
def tmp(y):
    global x
    x = x*y

tmp(2)
print(x)

24


In [91]:
def robot_factory( time, bp, orebot=1, ore=0, claybot=0, clay=0, obsbot=0, obs=0 ):
    '''This algo caps orebots at 3 so we don't waste time trying bad strats. This is not 100% guaranteed!'''

    max_ore_bots = 6 # tune speed (low) vs. thoroughness (high)
    global memo

    if time < 1:
        return 0
    key = ".".join([str(x) for x in [time,orebot,ore,claybot,clay,obsbot,obs]])
    if key in memo:
        return memo[key]

   # print(time, orebot, ore, claybot, clay, obsbot, obs)

    # Each branching point has four options: what to build next?
    best = 0

    ## Option 1: Build an Orebot ##
    ore_req = bp['orebot']['ore']
    ore_needed = max(ore_req-ore, 0)
    time_needed = 1 + ceil( ore_needed / orebot ) # 1 time to build bot. ceil(.) may be 0 if we already have enough ore

    # Is there enough time to make this orebot?
    # using strict because we need time to make the robot and also time for it to do a thing
    if orebot < max_ore_bots and time_needed < time:
        # How much ore is gained while we wait?
        # We may produce extra ore. e.g. if we need 1 but have 3 orebots.
        # Actually, we always produce extra ore because the orebot continues collecting while we build
        ore_gained = time_needed * orebot 

        # After waiting, we've gained some ore and spent some ore
        ore_left = ore + ore_gained - ore_req
        # We've also lost some time to collect ore and build the orebot
        time_left = time - time_needed

        # Plus we may gain other resources while we wait for ore
        clay_left = clay + claybot * time_needed
        obs_left = obs + obsbot * time_needed

        # Iterate, but we have one more orebot, and a different amount of ore/clay/obs/time
        gain = robot_factory( time_left, bp, orebot+1, ore_left, claybot, clay_left, obsbot, obs_left)
        best = max( best, gain )

    ## Option 2: Build a Claybot ##
    ore_req = bp['claybot']['ore']
    ore_needed = max(ore_req-ore, 0)
    time_needed = 1 + ceil( ore_needed / orebot ) 

    if time_needed < time:
        ore_gained = time_needed * orebot 
        ore_left = ore + ore_gained - ore_req
        time_left = time - time_needed

        clay_left = clay + claybot * time_needed
        obs_left = obs + obsbot * time_needed

        gain = robot_factory( time_left, bp,orebot, ore_left, claybot+1, clay_left, obsbot, obs_left )
        best = max( best, gain )

    ## Option 3: Build an Obsidianbot ##
    if claybot > 0:
        ore_req = bp['obsbot']['ore']
        clay_req = bp['obsbot']['clay']

        ore_needed = max(ore_req-ore, 0)
        clay_needed = max(clay_req-clay, 0)

        time_needed = 1 + max( ceil(ore_needed/orebot), ceil(clay_needed/claybot) )

        if time_needed < time:
            ore_left  =  ore + (time_needed *  orebot) -  ore_req
            clay_left = clay + (time_needed * claybot) - clay_req
            time_left = time - time_needed
            obs_left = obs + obsbot * time_needed

            gain = robot_factory( time_left, bp, orebot, ore_left, claybot, clay_left, obsbot+1, obs_left )
            best = max(best, gain)

    ## Option 4: Build a Geodebot ##
    if obsbot > 0:
        ore_req = bp['geobot']['ore']
        obs_req = bp['geobot']['obs']

        ore_needed = max(ore_req-ore, 0)
        obs_needed = max(obs_req-obs, 0)

        time_needed = 1 + max( ceil(ore_needed/orebot), ceil(obs_needed/obsbot) )

        if time_needed < time:
            ore_left = ore + (time_needed * orebot) - ore_req
            obs_left = obs + (time_needed * obsbot) - obs_req
            clay_left = clay + claybot * time_needed
            time_left = time - time_needed

            direct_gain = time_left # Gain 1 geode per time
            gain = direct_gain + robot_factory( time_left, bp, orebot, ore_left, claybot, clay_left, obsbot, obs_left )
            best = max(best, gain)

    memo[key] = best
    return best

In [93]:
memo = {}
robot_factory(24, test_bps[0])

9

In [95]:
memo = {}
robot_factory(24, test_bps[1])

12

In [83]:
out = []
# I think some of these bps don't let you get any geodes...
for bp in puz_bps:
    ans = robot_factory(24, bp)
    print(ans)
    out.append(ans)

4
0
12
1
0
7
3
0
1
0
9
0
1
0
2
3
12
1
2
3
13
10
2
0
1
1
2
4
0
0


With orebot capped at 3:
'4, 0, 12, 0, 0, 7, 2, 0, 1, 0, 8, 0, 1, 0, 2, 3, 12, 1, 1, 3, 13, 10, 1, 0, 0, 0, 2, 4, 0, 0'

With orebot capped at 6:
'4, 0, 12, 1, 0, 7, 3, 0, 1, 0, 9, 0, 1, 0, 2, 3, 12, 1, 2, 3, 13, 10, 2, 0, 1, 1, 2, 4, 0, 0'

In [ ]:
sum( [(i+1)*x for i,x in enumerate(out)] )

1382

### Part 2

In [97]:
memo = {}
robot_factory(32, test_bps[0])

KeyboardInterrupt: 

In [ ]:
memo = {}
robot_factory(32, test_bps[1])

In [ ]:
out = []
# I think some of these bps don't let you get any geodes...
for bp in puz_bps[:3]:
    memo = {}
    ans = robot_factory(32, bp)
    print(ans)
    out.append(ans)

KeyboardInterrupt: 

In [ ]:
out[0] * out[1] * out[2]